*Exploratory Data Analysis*

# Visualizing Depth Guided Sampling

In [1]:
import pickle
import matplotlib as mpl
from mpl_toolkits.mplot3d import Axes3D
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.patches import FancyArrowPatch, FancyArrow, ArrowStyle
from mpl_toolkits.mplot3d import proj3d
from matplotlib.lines import Line2D
import torch
import itertools
import warnings
warnings.filterwarnings("ignore")

from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

from run_dnerf_helpers import get_rays
from load_deepdeform import load_deepdeform_data, pose_spiral
from utils import Arrow3D, draw_transformed, draw_cam, draw_ray

%load_ext autoreload
%autoreload 2



Loading the validation logs.

In [2]:
path = "logs/human_06_mse"

sampling_strategy = "Hierarchical Sampling"
with open(os.path.join(path, "args.txt"), "r") as f:
    for line in f:
        if "use_depth_guided_sampling = True" in line:
            sampling_strategy = "Depth Guided Sampling"

ray_pickles = sorted([p for p in os.listdir(path) if ".pickle" in p])
frame_logs = []
for ray_pickle in ray_pickles:
    load_p = os.path.join(path, ray_pickle)
    with open(load_p, "rb") as f:
        print("Loading rays from " + load_p)
        frame_logs.append(pickle.load(f))

print("Logged items (shape/value):")
frame_log = frame_logs[0]
for k in frame_log.keys():
    toprint = "\t\t" + k
    if isinstance(frame_log[k], torch.Tensor):
        toprint += "\t" + str(tuple(frame_log[k].shape))
    if isinstance(frame_log[k], float):
        toprint += "\t" + str(frame_log[k])
    print(toprint)

Loading rays from logs/human_06_mse/step_050000_ray_debug.pickle
Loading rays from logs/human_06_mse/step_100000_ray_debug.pickle
Loading rays from logs/human_06_mse/step_150000_ray_debug.pickle
Loading rays from logs/human_06_mse/step_200000_ray_debug.pickle
Loading rays from logs/human_06_mse/step_250000_ray_debug.pickle
Loading rays from logs/human_06_mse/step_300000_ray_debug.pickle
Loading rays from logs/human_06_mse/step_350000_ray_debug.pickle
Loading rays from logs/human_06_mse/step_400000_ray_debug.pickle
Logged items (shape/value):
		rays_o	(240, 320, 3)
		rays_d	(240, 320, 3)
		rgb_map	(240, 320, 3)
		depth_map	(240, 320, 1)
		z_coarse	(240, 320, 64)
		z_fine	(240, 320, 128)
		c2w	(3, 4)
		near	0.10000000149011612
		far	3.708749771118164


In [3]:
cmap = [
    {"coarse": "#039BE5FF", "fine": "#0D47A1FF"},       # blue
    {"coarse": "#7CB342FF", "fine": "#33691EFF"},       # green
    {"coarse": "#FB8C00FF", "fine": "#E64A19FF"},       # orange
    {"coarse": "#BA68C8FF", "fine": "#4A148CFF"},       # purple
] 

def series(ith_frame):

    frame_log = frame_logs[ith_frame]
    
    H, W, _ = frame_log["rgb_map"].shape
    camera_pose = frame_log["c2w"]
    rays_o = frame_log["rays_o"]
    rays_d = frame_log["rays_d"]
    rgb_map = frame_log["rgb_map"]
    depth_map = frame_log["depth_map"]
    z_coarse = frame_log["z_coarse"]
    z_fine = frame_log["z_fine"]
    near = frame_log["near"]
    far = frame_log["far"]

    pts_coarse = rays_o[...,None,:] + rays_d[...,None,:] * z_coarse[...,:,None]
    pts_fine = rays_o[...,None,:] + rays_d[...,None,:] * z_fine[...,:,None]

    ith_rays = [(100, 100), (120, 150), (150, 200), (50, 270)]         # h, w

    fig = plt.figure(figsize=(10, 10))
    gs = fig.add_gridspec(2, 3)

    # Info
    info_str =  f"Exp.:  {path.split('/')[1]}\n"
    info_str += f"Step:  {ray_pickles[ith_frame].split('_')[1]}\n"
    info_str += f"Near: {near:.2f} m\n"
    info_str += f"Far:    {far:.2f} m"
    fig.text(x=0.13, y=0.5, s=info_str)

    # RGB img
    ax0 = fig.add_subplot(gs[0, 0])
    ax0.imshow(rgb_map)
    for i, ith_ray in enumerate(ith_rays):
        ax0.scatter(*ith_ray[::-1], fc=cmap[i]["coarse"], ec=cmap[i]["fine"], s=50)
    ax0.set_title("Predicted RGB Image")

    # Depth img
    ax0 = fig.add_subplot(gs[1, 0])
    dimg = ax0.imshow(depth_map)
    for i, ith_ray in enumerate(ith_rays):
        ax0.scatter(*ith_ray[::-1], fc=cmap[i]["coarse"], ec=cmap[i]["fine"], s=50)
    cax = fig.add_axes([0.1, 0.43, 0.2, 0.03])
    fig.colorbar(dimg, cax=cax, orientation='horizontal')
    ax0.set_title("Predicted Depth Map")

    # 3D rays plot
    ax1 = fig.add_subplot(gs[:, 1:], projection='3d')
    # ax1.computed_zorder = False
    ax1.set_title(f"{len(ith_rays)} Sampled Camera Rays")

    xlim = [-1, 1]
    ylim = [-1, 1]
    zlim = [-2, 2]
    ax1.set_xlabel('X')
    ax1.set_xlim(*xlim)
    ax1.set_ylabel('Y')
    ax1.set_ylim(*ylim)
    ax1.set_zlabel('Z')
    ax1.set_zlim(*zlim)
    ax1.set_box_aspect((xlim[1]-xlim[0], ylim[1]-ylim[0], zlim[1]-zlim[0]))       # -> length of 1 in each dimension is visually the equal

    # The world coordinate system
    axes_len = 0.5
    arrow_prop_dict = dict(mutation_scale=20, arrowstyle='simple', shrinkA=0, shrinkB=0)
    ax1.add_artist(Arrow3D([0, axes_len], [0, 0], [0, 0], **arrow_prop_dict, color='r'))
    ax1.add_artist(Arrow3D([0, 0], [0, axes_len], [0, 0], **arrow_prop_dict, color='b'))
    ax1.add_artist(Arrow3D([0, 0], [0, 0], [0, axes_len], **arrow_prop_dict, color='g'))
    ax1.text(-0.1, -0.1, 0, r'$0$')
    ax1.text(axes_len, 0, 0, r'$x$')
    ax1.text(0, axes_len, 0, r'$y$')
    ax1.text(0, 0, axes_len, r'$z$')

    # Draw the camera coordinate frame
    tcx, tcy, tcz, _ = draw_transformed(camera_pose, ax1, arrowstyle='simple', axes_len=0.5, linewidth=1.5, mutation_scale=20, edgecolor="black")
    # Draw the camera
    draw_cam(rays_o, rays_d, ax1, focal_dist=0.5)       # rays_o and rays_d are already in world-coordinates


    # Draw a ray with its samples
    for i, ith_ray in enumerate(ith_rays):
        draw_ray(rays_o[ith_ray], rays_d[ith_ray], pts_coarse[ith_ray], pts_fine[ith_ray], depth_map[ith_ray], 
                 near, far, ax=ax1, coarse_c=cmap[i]["coarse"], fine_c=cmap[i]["fine"])

    # Make the legends
    lgnd1 = plt.legend(handles=[tcx, tcy, tcz], 
            labels=["X", "Y", "Z"], 
            title="Camera pose", loc=1)
    plt.gca().add_artist(lgnd1)

    fig.suptitle(f"{sampling_strategy} in the Global Coordinate System", )
    fig.tight_layout()      
    
    return

inter = interact(series, ith_frame=(0, len(frame_logs)-1, 1))

interactive(children=(IntSlider(value=3, description='ith_frame', max=7), Output()), _dom_classes=('widget-int…